# Notebook 03: Optimal Model Validation

**Purpose**: Deep validation of optimal model through cross-validation, extrapolation, and long-term drift analysis

**Inputs**:
- `outputs/models/optimal_model.pkl` - Saved HarmonicAnalyzer from Notebook 02
- `outputs/csvs/optimal_model_params.csv` - Model parameters

**Outputs**:
- `outputs/csvs/cv_results.csv` - Cross-validation metrics
- `outputs/csvs/drift_analysis.csv` - Long-term drift data
- `outputs/csvs/extrapolation_predictions.csv` - Predictions -50k to +100k CE
- `outputs/csvs/residual_diagnostics.csv` - Diagnostic statistics
- Figures: CV performance, drift analysis, residual diagnostics

**Execution time**: ~4-5 minutes

---

## Key Questions

1. **Cross-validation**: Does the model generalize across different time periods?
2. **Extrapolation**: How accurate are predictions 50,000+ years beyond the data?
3. **Drift**: Does error accumulate (unbounded) or oscillate (bounded)?
4. **Precession scale**: What is the error at precession cycle boundaries (±25.7k, ±51.5k years)?
5. **Residuals**: Are they white noise or is there structure left?

## Setup & Load Optimal Model

In [1]:
import sys
sys.path.insert(0, '../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import swisseph as swe
import pickle
from scipy import stats

from src.enoch import enoch_calendar_frame, merge_astronomic_data
from src.harmonic_analysis import HarmonicAnalyzer

from src.enoch_config import SWISS_EPH_PATH
swe.set_ephe_path(SWISS_EPH_PATH)
swe.set_jpl_file('de441.eph')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Load optimal model
with open('outputs/models/optimal_model.pkl', 'rb') as f:
    analyzer = pickle.load(f)

# Load model parameters
params_df = pd.read_csv('outputs/csvs/optimal_model_params.csv')
n_harmonics = len(params_df) - 1  # -1 for offset row

print(f"✓ Loaded optimal {n_harmonics}-harmonic model")
print(f"  Periods: {params_df['period'].values[:-1]}")

✓ Loaded optimal 6-harmonic model
  Periods: [14700. 29400.  9800.  7350.  5880.  4900.]


## Load Full Dataset

In [2]:
YEAR_START = -12762
NUM_CYCLES = 100

e_df = enoch_calendar_frame(num_cycles=NUM_CYCLES)
a_df = merge_astronomic_data(e_df, YEAR_START, use_tt=True)

# Calculate solar year
a_df['solar_year'] = a_df['enoch_year'] + YEAR_START

day1_data = a_df[a_df.enoch_solar_doy == 1].copy()
day1_data = day1_data.sort_values('solar_year').reset_index(drop=True)

years = day1_data['solar_year'].values
ecliptic = day1_data['sun_ecliptic_longitude_neg'].values

valid_mask = ~np.isnan(ecliptic)
years = years[valid_mask]
ecliptic = ecliptic[valid_mask]

print(f"✓ Data loaded: {len(years):,} points ({years.min():.0f} to {years.max():.0f} CE)")

✓ Data loaded: 29,400 points (-12762 to 16636 CE)


## K-Fold Time Series Cross-Validation

In [3]:
print("Performing 5-Fold Expanding Window Cross-Validation...\n")

K_FOLDS = 5
cv_results = []

# Calculate fold boundaries
n_total = len(years)
initial_train_size = int(n_total * 0.5)  # Start with 50% for training
test_size = int((n_total - initial_train_size) / K_FOLDS)

for fold in range(K_FOLDS):
    # Expanding window: train size grows, test moves forward
    train_end_idx = initial_train_size + fold * test_size
    test_start_idx = train_end_idx
    test_end_idx = test_start_idx + test_size
    
    years_train_cv = years[:train_end_idx]
    ecliptic_train_cv = ecliptic[:train_end_idx]
    
    years_test_cv = years[test_start_idx:test_end_idx]
    ecliptic_test_cv = ecliptic[test_start_idx:test_end_idx]
    
    # Fit model on this fold's training data
    analyzer_cv = HarmonicAnalyzer(years_train_cv, ecliptic_train_cv)
    periods = params_df['period'].values[:-1]  # Exclude offset
    fit_cv = analyzer_cv.fit_multi_harmonic(periods.tolist())
    
    # Predict on test fold
    test_pred_cv = analyzer_cv.predict(years_test_cv)
    test_resid_cv = ecliptic_test_cv - test_pred_cv
    cv_rmse = np.sqrt(np.mean(test_resid_cv**2))
    cv_r2 = 1 - (np.sum(test_resid_cv**2) / np.sum((ecliptic_test_cv - np.mean(ecliptic_test_cv))**2))
    
    cv_results.append({
        'fold': fold + 1,
        'train_start': years_train_cv[0],
        'train_end': years_train_cv[-1],
        'test_start': years_test_cv[0],
        'test_end': years_test_cv[-1],
        'train_size': len(years_train_cv),
        'test_size': len(years_test_cv),
        'cv_rmse': cv_rmse,
        'cv_r2': cv_r2
    })
    
    print(f"Fold {fold+1}: Train [{years_train_cv[0]:.0f}, {years_train_cv[-1]:.0f}] → Test [{years_test_cv[0]:.0f}, {years_test_cv[-1]:.0f}]")
    print(f"  CV RMSE: {cv_rmse:.4f}°, CV R²: {cv_r2:.6f}")

cv_df = pd.DataFrame(cv_results)
mean_cv_rmse = cv_df['cv_rmse'].mean()
std_cv_rmse = cv_df['cv_rmse'].std()

print(f"\nCross-Validation Summary:")
print(f"  Mean CV RMSE: {mean_cv_rmse:.4f}° ± {std_cv_rmse:.4f}°")
print(f"  Mean CV R²: {cv_df['cv_r2'].mean():.6f}")

cv_df.to_csv('outputs/csvs/cv_results.csv', index=False)
print(f"\n✓ CV results saved: outputs/csvs/cv_results.csv")

Performing 5-Fold Expanding Window Cross-Validation...



/Users/meetri/dev/apps/python/enoch-manuscript/code/src/harmonic_analysis.py:122: RuntimeWarning: Detected irregular year spacing (99 anomalies, median step 1.000000). FFT results now use the median cadence; consider resampling for highest fidelity.
  warnings.warn(



=== Multi-Harmonic Fit Results ===
Fitted 6-frequency model
  Component 1: Period=   14700.0 yr, Amplitude= +0.915°, Phase= +1.932 rad
  Component 2: Period=   29400.0 yr, Amplitude= +1.112°, Phase= -0.011 rad
  Component 3: Period=    9800.0 yr, Amplitude= +0.200°, Phase= -0.371 rad
  Component 4: Period=    7350.0 yr, Amplitude= -0.038°, Phase= +0.553 rad
  Component 5: Period=    5880.0 yr, Amplitude= +0.001°, Phase= +0.741 rad
  Component 6: Period=    4900.0 yr, Amplitude= -0.002°, Phase= -0.304 rad
  Offset:  -0.164°

Goodness of fit:
  R² = 0.922746
  RMSE = 0.282°
Fold 1: Train [-12762, 1936] → Test [1938, 4876]
  CV RMSE: 0.2948°, CV R²: 0.148405

=== Multi-Harmonic Fit Results ===
Fitted 6-frequency model
  Component 1: Period=   14700.0 yr, Amplitude= +0.931°, Phase= +2.084 rad
  Component 2: Period=   29400.0 yr, Amplitude= +1.300°, Phase= -0.103 rad
  Component 3: Period=    9800.0 yr, Amplitude= +0.223°, Phase= -0.051 rad
  Component 4: Period=    7350.0 yr, Amplitude= -

/Users/meetri/dev/apps/python/enoch-manuscript/code/src/harmonic_analysis.py:122: RuntimeWarning: Detected irregular year spacing (119 anomalies, median step 1.000000). FFT results now use the median cadence; consider resampling for highest fidelity.
  warnings.warn(
/Users/meetri/dev/apps/python/enoch-manuscript/code/src/harmonic_analysis.py:122: RuntimeWarning: Detected irregular year spacing (139 anomalies, median step 1.000000). FFT results now use the median cadence; consider resampling for highest fidelity.
  warnings.warn(



=== Multi-Harmonic Fit Results ===
Fitted 6-frequency model
  Component 1: Period=   14700.0 yr, Amplitude= +0.854°, Phase= +1.810 rad
  Component 2: Period=   29400.0 yr, Amplitude= +1.054°, Phase= +0.088 rad
  Component 3: Period=    9800.0 yr, Amplitude= -0.183°, Phase= +2.289 rad
  Component 4: Period=    7350.0 yr, Amplitude= +0.052°, Phase= +2.610 rad
  Component 5: Period=    5880.0 yr, Amplitude= -0.016°, Phase= +2.951 rad
  Component 6: Period=    4900.0 yr, Amplitude= +0.004°, Phase= +3.298 rad
  Offset:  -0.219°

Goodness of fit:
  R² = 0.922812
  RMSE = 0.285°
Fold 3: Train [-12762, 7816] → Test [7818, 10756]
  CV RMSE: 0.2914°, CV R²: 0.313217

=== Multi-Harmonic Fit Results ===
Fitted 6-frequency model
  Component 1: Period=   14700.0 yr, Amplitude= -0.880°, Phase= -1.315 rad
  Component 2: Period=   29400.0 yr, Amplitude= +1.065°, Phase= +0.055 rad
  Component 3: Period=    9800.0 yr, Amplitude= +0.204°, Phase= -0.809 rad
  Component 4: Period=    7350.0 yr, Amplitude= 

/Users/meetri/dev/apps/python/enoch-manuscript/code/src/harmonic_analysis.py:122: RuntimeWarning: Detected irregular year spacing (159 anomalies, median step 1.000000). FFT results now use the median cadence; consider resampling for highest fidelity.
  warnings.warn(
/Users/meetri/dev/apps/python/enoch-manuscript/code/src/harmonic_analysis.py:122: RuntimeWarning: Detected irregular year spacing (179 anomalies, median step 1.000000). FFT results now use the median cadence; consider resampling for highest fidelity.
  warnings.warn(



=== Multi-Harmonic Fit Results ===
Fitted 6-frequency model
  Component 1: Period=   14700.0 yr, Amplitude= -1.090°, Phase= -1.323 rad
  Component 2: Period=   29400.0 yr, Amplitude= +1.101°, Phase= -0.155 rad
  Component 3: Period=    9800.0 yr, Amplitude= +0.373°, Phase= -1.016 rad
  Component 4: Period=    7350.0 yr, Amplitude= -0.196°, Phase= -0.933 rad
  Component 5: Period=    5880.0 yr, Amplitude= +0.117°, Phase= -0.892 rad
  Component 6: Period=    4900.0 yr, Amplitude= -0.069°, Phase= -0.879 rad
  Offset:  -0.081°

Goodness of fit:
  R² = 0.933613
  RMSE = 0.289°
Fold 5: Train [-12762, 13696] → Test [13698, 16636]
  CV RMSE: 2.7281°, CV R²: -13.769836

Cross-Validation Summary:
  Mean CV RMSE: 0.8546° ± 1.0568°
  Mean CV R²: -2.692090

✓ CV results saved: outputs/csvs/cv_results.csv


## Long-Term Extrapolation & Drift Analysis

In [4]:
print("Generating long-term extrapolation (-50,000 to +100,000 CE)...\n")

# Generate prediction years
extrap_years = np.arange(-50000, 100001, 100)  # Every 100 years
extrap_predictions = analyzer.predict(extrap_years)

# Calculate drift metrics at key horizons
PRECESSION_PERIOD = 25772
data_end = years.max()

# Precession cycle boundaries relative to data end
boundaries = [0, PRECESSION_PERIOD, 2*PRECESSION_PERIOD, 3*PRECESSION_PERIOD, -PRECESSION_PERIOD, -2*PRECESSION_PERIOD]

drift_analysis = []
for boundary_offset in boundaries:
    target_year = data_end + boundary_offset
    if target_year < extrap_years.min() or target_year > extrap_years.max():
        continue
    
    # Find closest prediction
    idx = np.argmin(np.abs(extrap_years - target_year))
    pred_value = extrap_predictions[idx]
    
    drift_analysis.append({
        'time_horizon': boundary_offset,
        'year': extrap_years[idx],
        'predicted_value': pred_value,
        'abs_predicted_value': np.abs(pred_value)
    })

drift_df = pd.DataFrame(drift_analysis)

# Calculate drift rate (linear regression on absolute predictions vs time)
from scipy.stats import linregress
slope, intercept, r_value, p_value, std_err = linregress(
    drift_df['time_horizon'], 
    drift_df['abs_predicted_value']
)

drift_rate_per_millennium = slope * 1000  # Convert to °/millennium

print(f"Drift Analysis:")
print(f"  Drift rate: {drift_rate_per_millennium:.6f} °/millennium")
print(f"  p-value: {p_value:.6f} ({'significant' if p_value < 0.05 else 'not significant'})")
print(f"  Error at precession boundaries:")
for _, row in drift_df.iterrows():
    print(f"    {row['time_horizon']:+7.0f} yr: {row['predicted_value']:+7.2f}° (|{row['abs_predicted_value']:.2f}°|)")

# Save extrapolation predictions
extrap_df = pd.DataFrame({
    'year': extrap_years,
    'predicted_longitude': extrap_predictions
})
extrap_df.to_csv('outputs/csvs/extrapolation_predictions.csv', index=False)
print(f"\n✓ Extrapolation predictions saved: outputs/csvs/extrapolation_predictions.csv")

drift_df.to_csv('outputs/csvs/drift_analysis.csv', index=False)
print(f"✓ Drift analysis saved: outputs/csvs/drift_analysis.csv")

Generating long-term extrapolation (-50,000 to +100,000 CE)...

Drift Analysis:
  Drift rate: -0.013152 °/millennium
  p-value: 0.028431 (significant)
  Error at precession boundaries:
         +0 yr:   -0.51° (|0.51°|)
     +25772 yr:   +0.98° (|0.98°|)
     +51544 yr:   +0.36° (|0.36°|)
     +77316 yr:   -0.16° (|0.16°|)
     -25772 yr:   -1.98° (|1.98°|)
     -51544 yr:   -1.66° (|1.66°|)

✓ Extrapolation predictions saved: outputs/csvs/extrapolation_predictions.csv
✓ Drift analysis saved: outputs/csvs/drift_analysis.csv


## Residual Diagnostics

In [5]:
# Get residuals from optimal model fit on training data
# (Refit if needed or extract from saved model)
fitted_values = analyzer.predict(analyzer.years)
residuals = analyzer.values - fitted_values

# Compute diagnostics
diagnostics = {
    'residual_mean': np.mean(residuals),
    'residual_std': np.std(residuals),
    'residual_min': np.min(residuals),
    'residual_max': np.max(residuals),
    'residual_range': np.max(residuals) - np.min(residuals),
    'residual_skewness': stats.skew(residuals),
    'residual_kurtosis': stats.kurtosis(residuals),
    'normality_pvalue': stats.normaltest(residuals).pvalue,
    'autocorr_lag1': np.corrcoef(residuals[:-1], residuals[1:])[0, 1]
}

diag_df = pd.DataFrame([diagnostics])
diag_df.to_csv('outputs/csvs/residual_diagnostics.csv', index=False)

print("Residual Diagnostics:")
print(f"  Mean: {diagnostics['residual_mean']:.6f}° (should be ~0)")
print(f"  Std: {diagnostics['residual_std']:.4f}°")
print(f"  Range: [{diagnostics['residual_min']:.2f}°, {diagnostics['residual_max']:.2f}°]")
print(f"  Normality p-value: {diagnostics['normality_pvalue']:.6f}")
print(f"  Autocorrelation (lag-1): {diagnostics['autocorr_lag1']:.4f}")
print(f"\n✓ Residual diagnostics saved: outputs/csvs/residual_diagnostics.csv")

Residual Diagnostics:
  Mean: 0.000000° (should be ~0)
  Std: 0.2029°
  Range: [-0.65°, 0.64°]
  Normality p-value: 0.000000
  Autocorrelation (lag-1): -0.2657

✓ Residual diagnostics saved: outputs/csvs/residual_diagnostics.csv


## Summary

In [6]:
print("="*70)
print("OPTIMAL MODEL VALIDATION SUMMARY")
print("="*70)

print(f"\nModel: {n_harmonics}-harmonic optimal model")

print(f"\nCross-Validation (K={K_FOLDS}):")
print(f"  Mean CV RMSE: {mean_cv_rmse:.4f}° ± {std_cv_rmse:.4f}°")
print(f"  Stability: {'Excellent' if std_cv_rmse < 0.1 else 'Good' if std_cv_rmse < 0.3 else 'Moderate'}")

print(f"\nLong-Term Drift:")
print(f"  Drift rate: {drift_rate_per_millennium:.6f} °/millennium")
print(f"  Statistical significance: p={p_value:.6f} ({'Yes' if p_value < 0.05 else 'No'})")
print(f"  Behavior: {'Bounded oscillation' if abs(drift_rate_per_millennium) < 0.01 else 'Possible secular drift'}")

print(f"\nPrecession-Scale Error:")
max_error_at_boundaries = drift_df['abs_predicted_value'].max()
print(f"  Maximum error at boundaries: {max_error_at_boundaries:.2f}°")
print(f"  Within ±15° bounds: {'Yes ✓' if max_error_at_boundaries <= 15 else 'No ✗'}")

print(f"\nResidual Quality:")
print(f"  White noise test: {'Pass ✓' if diagnostics['normality_pvalue'] > 0.05 else 'Fail ✗'} (p={diagnostics['normality_pvalue']:.4f})")
print(f"  Autocorrelation: {diagnostics['autocorr_lag1']:.4f} ({'Low' if abs(diagnostics['autocorr_lag1']) < 0.3 else 'Moderate'})")

print("\n" + "="*70)
print("✓ Notebook 03 complete")
print("="*70)
print(f"\nNext step: Run Notebook 04 (294-Multiple Resonance)")

OPTIMAL MODEL VALIDATION SUMMARY

Model: 6-harmonic optimal model

Cross-Validation (K=5):
  Mean CV RMSE: 0.8546° ± 1.0568°
  Stability: Moderate

Long-Term Drift:
  Drift rate: -0.013152 °/millennium
  Statistical significance: p=0.028431 (Yes)
  Behavior: Possible secular drift

Precession-Scale Error:
  Maximum error at boundaries: 1.98°
  Within ±15° bounds: Yes ✓

Residual Quality:
  White noise test: Fail ✗ (p=0.0000)
  Autocorrelation: -0.2657 (Low)

✓ Notebook 03 complete

Next step: Run Notebook 04 (294-Multiple Resonance)
